In [71]:
import pdfplumber
import pandas as pd
import pprint
import re
import os
import datetime

from etl.log import get_logger

logger = get_logger

def build_date(filename):
    global logger
    name_parts = filename.split("_")
    if len(name_parts) >= 4:
        sales_year = int(name_parts[2])
        sales_day = int(name_parts[3])
        sales_month = int(name_parts[4])

        if sales_month > 12:
            sales_day = int(name_parts[4])
            sales_month = int(name_parts[3])

        if sales_day > 31:
            sales_day = int(name_parts[3][0:2])
    else:
        logger.info("filename is not formatted correctly")
        return None

    return datetime.datetime(sales_year, sales_month, sales_day)


In [72]:
FOLDER = "data"
EXTENSION_PDFS = [".pdf"]
REGEX = r"(\w+)\s(\$\s\S+)\s(\$\s\S+)\s(\$\s\S+)\s(\$\s\S+)\s(\$\s\S+)\s(\$\s\S+)\s(\$\s\S+)"

In [79]:

for file in os.listdir(FOLDER):
  filename, ext = os.path.splitext(file)
  if ext in EXTENSION_PDFS:
    sales_date = build_date(filename)
    pdf = pdfplumber.open(os.path.join(FOLDER, file))
    data = []
    volumes = []
    for page in pdf.pages:
      rows = page.extract_text_lines()
      orig_record = {}
      for item in rows:
        if 'Dispenser' in item['text']:
          orig_record = {
            "dispenser": item['text'].strip(),
            "sales_date": sales_date
          }
        else:
          fuel_type = item['text'].split(' ')[0]
          if fuel_type in ['unleaded','Premium','Diesel']:
            matches = re.finditer(REGEX, item['text'], re.MULTILINE)
            mat_list = list(matches)
            if len(mat_list) > 0:
              for idx, match in enumerate(mat_list, start=1):
                values = list(match.groups())
                record = orig_record.copy()
                record["fuel_type"] = values[0]
                record["fuel_sales"] = values[1]
                record["fuel_sales_discount"] = values[3]
                record["fuel_sales_paid"] = values[6]          
                data.append(record)
            else:
              volumes.append(item['text'])
      
      if len(volumes) > 0:
        for idx, item in enumerate(volumes):
          if idx < len(data):
            values = item.split(' ')
            if len(values) == 5:
              data[idx]["fuel_volume"] = values[4]
      
      print(len(data))
      raw_df = pd.DataFrame(data)
      raw_df.to_csv(os.path.join(FOLDER,filename+'.csv'),index=False)
      
raw_df

9
9
12
12


,dispenser,sales_date,fuel_type,fuel_sales,fuel_sales_discount,fuel_sales_paid,fuel_volume
0,Dispenser Id 1,2023-08-13,unleaded,"$ 1,148.89",$ -6.76,"$ 1,142.13",313.130
1,Dispenser Id 1,2023-08-13,Premium,$ 228.02,$ 0.00,$ 228.02,55.900
2,Dispenser Id 1,2023-08-13,Diesel,$ 532.30,$ -0.62,$ 531.68,134.800
3,Dispenser Id 2,2023-08-13,unleaded,$ 330.52,$ -0.28,$ 330.24,90.090
4,Dispenser Id 2,2023-08-13,Premium,$ 146.05,$ -1.13,$ 144.92,35.800
5,Dispenser Id 2,2023-08-13,Diesel,$ 9.84,$ 0.00,$ 9.84,2.490
6,Dispenser Id 3,2023-08-13,unleaded,"$ 1,159.66",$ -5.31,"$ 1,154.35",316.060
7,Dispenser Id 3,2023-08-13,Premium,$ 109.43,$ 0.00,$ 109.43,26.820
8,Dispenser Id 3,2023-08-13,Diesel,$ 104.18,$ 0.00,$ 104.18,26.380
9,Dispenser Id 4,2023-08-13,unleaded,$ 572.61,$ -3.49,$ 569.12,156.070
